# Training Analysis Project For Code Louisville Summer 2020 Program

## Import pandas and CSV file
Import pandas and read the Samplecomments2 CSV file, storing it in df

In [1]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show, reset_output
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from scipy import stats

In [2]:
training_df = pd.read_csv('Samplecomments2.csv')
restaurant_df = pd.read_csv('Restaurantdata.csv')

## Take a look at the data in the CSV Files
Use: 
- .head to see the data in a spreadsheet format (put 10 in the parentheses to see all 10 rows)
- .shape to see how many rows/columns there are

Determine where there's a matching individual identifier to combine both DataFrames.

In [3]:
training_df.head(10)

,Username,Date,Restaurant,OSC,Receiving,Wish,Feedback
0,ABC123,5/20/2020,B000001,4.00,Yes,Shorter; More shoulder-to-shoulder,I love the training!
1,ABC234,5/20/2020,B000002,3.20,No,More shoulder-to-shoulder,The training is alright.
2,ABC345,5/20/2020,B000003,3.80,Yes,More computer,Training is okay.
3,ABC456,5/20/2020,B000004,2.80,No,Longer,Training is terrible.
4,ABC567,5/19/2020,B000005,2.50,No,Shorter; More shoulder-to-shoulder,I hate the training.
5,ABC678,5/19/2020,B000006,3.10,No,Shorter; More computer,I wish I didn't have to take the training.
6,ABC789,5/19/2020,B000007,3.60,Yes,Shorter,The training is pretty good.
7,ABC890,5/15/2020,B000008,3.80,Yes,More shoulder-to-shoulder,The training is really helpful.
8,ABC901,5/15/2020,B000009,3.95,Yes,Longer; More shoulder-to-shoulder,Training helps my team and I perform our jobs ...
9,ABC902,5/14/2020,B000010,2.50,No,More computer,Training is a waste of time and I don't want i...


In [4]:
training_df.shape

(10, 7)

In [5]:
restaurant_df.head()

,Restaurant,SalesMil,Safety,WindowSeconds,BestofBestAward,FullyTrained,Staff,FollowSchedule
0,B000001,$2.2,Yes,65,Yes,Yes,40,Yes
1,B000002,$2.1,Yes,80,No,Yes,36,Yes
2,B000003,$2.0,Yes,75,Yes,Yes,38,Yes
3,B000004,$1.6,Yes,120,No,Yes,17,No
4,B000005,$1.2,No,130,No,No,13,No


In [6]:
restaurant_df.shape

(20, 8)

## Combine DataFrames

Join the two DataFrames (training_df & restaurant_df) into a new one for analysis purposes. Restaurant_df contains more data, but we only care about looking at restaurants that completed the training survey (training_df), so we will want to do a left join. 

Convert Wish & Feedback columns to lowercase to avoid errors later on.

In [7]:
complete_df = pd.merge(left=training_df, right=restaurant_df, how='left', left_on='Restaurant', right_on='Restaurant')

In [8]:
complete_df['Wish'] = complete_df['Wish'].str.lower()
complete_df['Feedback'] = complete_df['Feedback'].str.lower()

In [9]:
complete_df.head(10)

,Username,Date,Restaurant,OSC,Receiving,Wish,Feedback,SalesMil,Safety,WindowSeconds,BestofBestAward,FullyTrained,Staff,FollowSchedule
0,ABC123,5/20/2020,B000001,4.00,Yes,shorter; more shoulder-to-shoulder,i love the training!,$2.2,Yes,65,Yes,Yes,40,Yes
1,ABC234,5/20/2020,B000002,3.20,No,more shoulder-to-shoulder,the training is alright.,$2.1,Yes,80,No,Yes,36,Yes
2,ABC345,5/20/2020,B000003,3.80,Yes,more computer,training is okay.,$2.0,Yes,75,Yes,Yes,38,Yes
3,ABC456,5/20/2020,B000004,2.80,No,longer,training is terrible.,$1.6,Yes,120,No,Yes,17,No
4,ABC567,5/19/2020,B000005,2.50,No,shorter; more shoulder-to-shoulder,i hate the training.,$1.2,No,130,No,No,13,No
5,ABC678,5/19/2020,B000006,3.10,No,shorter; more computer,i wish i didn't have to take the training.,$1.1,Yes,100,No,Yes,18,No
6,ABC789,5/19/2020,B000007,3.60,Yes,shorter,the training is pretty good.,$1.9,Yes,90,No,Yes,19,No
7,ABC890,5/15/2020,B000008,3.80,Yes,more shoulder-to-shoulder,the training is really helpful.,$1.8,Yes,70,No,Yes,22,Yes
8,ABC901,5/15/2020,B000009,3.95,Yes,longer; more shoulder-to-shoulder,training helps my team and i perform our jobs ...,$1.8,Yes,75,Yes,Yes,37,Yes
9,ABC902,5/14/2020,B000010,2.50,No,more computer,training is a waste of time and i don't want i...,$1.1,No,180,No,No,17,No


In [10]:
complete_df.shape

(10, 14)

## Clean up data

- Convert date column into date data
- Convert SalesMil column into float (and remove $)
- Gut check the data types and full Data Frame to make sure conversions happen

In [11]:
complete_df['Date'] = complete_df['Date'].astype('datetime64[ns]')
complete_df['Date'] = complete_df['Date'].dt.date

In [12]:
complete_df['SalesMil'] = complete_df['SalesMil'].replace({'\$':''}, regex = True)
complete_df['SalesMil'] = complete_df['SalesMil'].astype(str).astype(float)

In [13]:
complete_df.dtypes

Username            object
Date                object
Restaurant          object
OSC                float64
Receiving           object
Wish                object
Feedback            object
SalesMil           float64
Safety              object
WindowSeconds        int64
BestofBestAward     object
FullyTrained        object
Staff                int64
FollowSchedule      object
dtype: object

In [14]:
complete_df.head()

,Username,Date,Restaurant,OSC,Receiving,Wish,Feedback,SalesMil,Safety,WindowSeconds,BestofBestAward,FullyTrained,Staff,FollowSchedule
0,ABC123,2020-05-20,B000001,4.0,Yes,shorter; more shoulder-to-shoulder,i love the training!,2.2,Yes,65,Yes,Yes,40,Yes
1,ABC234,2020-05-20,B000002,3.2,No,more shoulder-to-shoulder,the training is alright.,2.1,Yes,80,No,Yes,36,Yes
2,ABC345,2020-05-20,B000003,3.8,Yes,more computer,training is okay.,2.0,Yes,75,Yes,Yes,38,Yes
3,ABC456,2020-05-20,B000004,2.8,No,longer,training is terrible.,1.6,Yes,120,No,Yes,17,No
4,ABC567,2020-05-19,B000005,2.5,No,shorter; more shoulder-to-shoulder,i hate the training.,1.2,No,130,No,No,13,No


## Look at how people responded to the "Receiving" question
Take a look at how many people said they are receiving what they need to train their people on promotions versus the number of people who do not. 

"Normalized" the counts to represent as a decimal/percentage for both yes/no. Leaving "normalize" off displays the actual count of each response.

In [15]:
complete_df['Receiving'].value_counts(normalize=True)

No     0.5
Yes    0.5
Name: Receiving, dtype: float64

## Create new functions based on response to "Receiving" question

Create a new function using a boolean to sort and show only those who said they *do not* receive the training they need. Then, check how many records appear, to gut check it's correct.

Repeat for those who said they *do* receive the training they need.

In [16]:
said_no = complete_df[complete_df['Receiving'] == 'No']

In [17]:
said_no.shape

(5, 14)

In [18]:
said_yes = complete_df[complete_df['Receiving'] == 'Yes']

In [19]:
said_yes.shape

(5, 14)

## Examine the OSC scores compared to the "Receiving" response
Print:
- Average OSC score of those who said no vs. yes
- Median OSC score of no vs.  yes

In [20]:
print(said_no['OSC'].mean(),
      said_yes['OSC'].mean(),
      said_no['OSC'].median(),
      said_yes['OSC'].median()
     )

2.82 3.8299999999999996 2.8 3.8


These results show those who say they are *not* receiving the training they need have lower average and median OSC scores. To continue analysis, create two new functions to group those who have a OSC score greater than 3 and those who have 3 or less.

In [21]:
over3 = complete_df[complete_df['OSC'] > 3]
undereq3 = complete_df[complete_df['OSC'] <= 3]

Take a look at the percentage of yes/no responses for those with scores over 3 vs. under or equal to 3.

In [22]:
print(over3['Receiving'].value_counts(normalize=True))

Yes    0.714286
No     0.285714
Name: Receiving, dtype: float64


In [23]:
print(undereq3['Receiving'].value_counts(normalize=True))

No    1.0
Name: Receiving, dtype: float64


These results show for those with OSC scores *above* 3, the majority are getting what they need. For those with scores *under or equal to* 3, 100% of them do not feel they're receiving the training they need.

Take a look at the sample sizes to make sure it's a fair distribution:

In [24]:
print(len(over3))
print(len(undereq3))

7
3


## Surface-Level Analysis on Sales, Window Speed, & Staff Counts as related to "Receiving"

In [25]:
print(said_no['SalesMil'].mean(),
      said_yes['SalesMil'].mean(),
      said_no['SalesMil'].median(),
      said_yes['SalesMil'].median()
     )

1.42 1.94 1.2 1.9


Results show those who do not feel they are receiving the training they need have lower sales.

In [26]:
print(said_no['WindowSeconds'].mean(),
      said_yes['WindowSeconds'].mean(),
      said_no['WindowSeconds'].median(),
      said_yes['WindowSeconds'].median()
     )

122.0 75.0 120.0 75.0


Results show those who do not feel they are receiving the training they need have slower window speeds.

In [27]:
print(said_no['Staff'].mean(),
      said_yes['Staff'].mean(),
      said_no['Staff'].median(),
      said_yes['Staff'].median()
     )

20.2 31.2 17.0 37.0


Results show those who do not feel they are receiving the training they need have fewer staff (which, logically, may be contributing to their reduced performance as well).

## Look at responses to the "Wish" question
Keeping in mind that people can select multiple wishes, analyze:
- How many people selected "shorter" training
- Split up every entry (split by ; in csv file) and stack so each individual item can be counted; then, check the count for each to find most popular wishes

In [28]:
wishshorter = complete_df['Wish'].str.contains('shorter')
wishshorter.value_counts(normalize=True)

False    0.6
True     0.4
Name: Wish, dtype: float64

In [29]:
wishlist = complete_df["Wish"].str.split(';', expand=True)
wishlist.stack().value_counts()

shorter                       4
 more shoulder-to-shoulder    3
more computer                 2
longer                        2
more shoulder-to-shoulder     2
 more computer                1
dtype: int64

## Tests of Statistical Significance

### Correlations between OSC, Window Speed, and Staff

In [30]:
complete_df.corr(method ='pearson')

,OSC,SalesMil,WindowSeconds,Staff
OSC,1.000000,0.777464,-0.885202,0.743057
SalesMil,0.777464,1.000000,-0.794241,0.794265
WindowSeconds,-0.885202,-0.794241,1.000000,-0.690236
Staff,0.743057,0.794265,-0.690236,1.000000


### T-Tests Examining Receiving Response vs. Window Speeds, Sales, and Staff

In [31]:
group1 = complete_df[complete_df['Receiving'] == 'Yes']['WindowSeconds']

group2= complete_df[complete_df['Receiving'] == 'No']['WindowSeconds']

stats.ttest_ind(group2, group1)

Ttest_indResult(statistic=2.706787733488262, pvalue=0.02679098799809847)

In [32]:
group1 = complete_df[complete_df['Receiving'] == 'Yes']['SalesMil']

group2= complete_df[complete_df['Receiving'] == 'No']['SalesMil']

stats.ttest_ind(group2, group1)

Ttest_indResult(statistic=-2.507662675286143, pvalue=0.03650361140303531)

In [33]:
group1 = complete_df[complete_df['Receiving'] == 'Yes']['Staff']

group2= complete_df[complete_df['Receiving'] == 'No']['Staff']

stats.ttest_ind(group2, group1)

Ttest_indResult(statistic=-1.836396549058989, pvalue=0.10361628417712324)

### T-Tests Examining Fully Trained vs. Window Speeds, Sales, and Staff

In [34]:
group1 = complete_df[complete_df['FullyTrained'] == 'Yes']['WindowSeconds']

group2= complete_df[complete_df['FullyTrained'] == 'No']['WindowSeconds']

stats.ttest_ind(group2, group1)

Ttest_indResult(statistic=4.227808617272896, pvalue=0.002884958033543161)

In [35]:
group1 = complete_df[complete_df['FullyTrained'] == 'Yes']['SalesMil']

group2= complete_df[complete_df['FullyTrained'] == 'No']['SalesMil']

stats.ttest_ind(group2, group1)

Ttest_indResult(statistic=-2.5958102558849947, pvalue=0.031824373353203754)

In [36]:
group1 = complete_df[complete_df['FullyTrained'] == 'Yes']['Staff']

group2= complete_df[complete_df['FullyTrained'] == 'No']['Staff']

stats.ttest_ind(group2, group1)

Ttest_indResult(statistic=-1.7663946018051353, pvalue=0.11531802055555798)

# Visualizations

In [37]:
output_notebook()

p = figure(
    title="OSC and Staff Trends", 
    plot_width=400, 
    plot_height=400, 
    x_axis_label='Staff Count', 
    y_axis_label = 'OSC Score'
)

x_values = complete_df['Staff']
y_values = complete_df['OSC']

# add a circle renderer with a size & color (KFC Red)
p.circle(x=x_values, y=y_values, size=20, color="#e4002b")

# show the results
show(p)


Loading BokehJS ...

In [38]:
output_notebook()

p = figure(
    title="OSC and Staff Trends", 
    plot_width=400, 
    plot_height=400, 
    x_axis_label='Staff Count', 
    y_axis_label = 'OSC Score'
)

x_values = complete_df['Staff']
y_values = complete_df['SalesMil']

# add a circle renderer with a size & color
p.circle(x=x_values, y=y_values, size=20, color="gray")

# show the results
show(p)


Loading BokehJS ...

In [39]:
output_notebook()

p = figure(
    title="OSC and Staff Trends", 
    plot_width=400, 
    plot_height=400, 
    x_axis_label='Staff Count', 
    y_axis_label = 'OSC Score'
)

x_values = complete_df['Staff']
y_values = complete_df['WindowSeconds']

# add a circle renderer with a size & color (KFC Red)
p.circle(x=x_values, y=y_values, size=20, color="#000000")

# show the results
show(p)


Loading BokehJS ...

In [59]:
#Trying to display bar graph of the yes/no receiving question

reset_output
output_notebook()

feedback = complete_df.groupby(['Receiving']).mean()

source = ColumnDataSource(feedback)

p = figure(x_range=feedback, plot_height=250, title="Receiving Training Responses",
           toolbar_location=None, tools="")

p.vbar(x=Response, top=Counts, width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)


Loading BokehJS ...

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().